In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost -q

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [ ]:
%%time
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']
  
  path_0 = '/content/drive/My Drive/Zimnat/stack/'
  path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
  path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'
  submit1 = pd.read_csv(path_0 + '0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv( path_1 +'1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv(path_2 + '2_fold' + str(r) + '.csv')

  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')

  test.columns = ['target', 'ID X PCODE', 'Label1', 'Label2', 'Label3']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

CPU times: user 25.4 s, sys: 901 ms, total: 26.3 s
Wall time: 36.5 s


In [ ]:
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

path_0 = '/content/drive/My Drive/Zimnat/stack/'
path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'

submit1 = pd.read_csv(path_0+ '0_main.csv')
submit2 = pd.read_csv(path_1 + '1_main.csv')
submit3 = pd.read_csv(path_2+ '2_main.csv')

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1', 'Label2', 'Label3']

In [ ]:
fold_train.head()

,target,ID X PCODE,Label1,Label2,Label3
0,0,0_0 X P5DA,0.000002,0.000007,0.000104
1,0,0_0 X RIBP,0.000040,0.000067,0.001975
2,0,0_0 X 8NN1,0.000001,0.000004,0.000066
3,0,0_0 X 7POT,0.000007,0.000003,0.000071
4,0,0_0 X 66FJ,0.000007,0.000003,0.000082


In [ ]:
test.head()

,ID X PCODE,Label1,Label2,Label3
0,F86J5PC X P5DA,0.000002,0.000106,0.000351
1,F86J5PC X RIBP,0.000395,0.000344,0.002852
2,F86J5PC X 8NN1,0.000003,0.000065,0.000044
3,F86J5PC X 7POT,0.000071,0.000061,0.000097
4,F86J5PC X 66FJ,0.000073,0.000050,0.000036


new

In [ ]:
fold_train['ID'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
fold_train['PCODE'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

test['ID'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
test['PCODE'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

del fold_train['ID X PCODE'], test['ID X PCODE']

In [ ]:
new_df = pd.DataFrame(np.zeros((fold_train['ID'].nunique(), fold_train['PCODE'].nunique()*3+1)))
new_df.index = fold_train['ID'].unique()
new_df.columns = list(fold_train['PCODE'].unique() + '_0') + list(fold_train['PCODE'].unique() + '_1') + list(fold_train['PCODE'].unique() + '_2') + ['target']

In [16]:
for v in tqdm(fold_train.values):
   new_df.loc[v[4], v[5]+'_0'] = v[1]
   new_df.loc[v[4], v[5]+'_1'] = v[2]
   new_df.loc[v[4], v[5]+'_2'] = v[3]
   if v[0] == 1:
     new_df.loc[v[4], 'target'] = v[5]

100%|██████████| 1298055/1298055 [51:44<00:00, 418.10it/s]


In [17]:
new_df

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,RVSZ_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,K6QO_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,RVSZ_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,K6QO_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,RVSZ_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,K6QO_2,target
0_0,2.062916e-06,0.000040,1.329303e-06,0.000007,0.000007,1.562166e-07,0.000004,0.999314,0.000062,0.000012,0.000108,0.000050,0.000030,9.034984e-07,0.000090,0.000008,0.000020,0.000016,0.000182,0.000037,0.000000,0.000007,0.000067,0.000004,0.000003,0.000003,0.000005,0.000007,0.999873,0.000066,0.000022,0.000036,0.000011,0.000010,0.000005,0.000160,0.000036,0.000022,0.000016,0.000163,0.000031,0.000000,0.000104,0.001975,0.000066,0.000071,0.000082,0.000173,0.000069,0.975555,0.001077,0.000423,0.001417,0.000504,0.000250,0.000041,0.007239,0.000717,0.007934,0.005173,0.006053,0.000378,0.000000,RVSZ
0_1,1.194800e-05,0.000173,3.812930e-06,0.000010,0.000021,3.163981e-07,0.000012,0.000000,0.000614,0.000043,0.000445,0.000158,0.000069,2.675490e-06,0.003556,0.000186,0.000047,0.000042,0.002238,0.000797,0.991568,0.000084,0.000554,0.000027,0.000027,0.000023,0.000041,0.000052,0.000000,0.001033,0.000398,0.000613,0.000078,0.000076,0.000056,0.004113,0.000281,0.000222,0.000148,0.004045,0.000471,0.997128,0.000104,0.001975,0.000066,0.000071,0.000082,0.000173,0.000069,0.000000,0.001077,0.000423,0.001417,0.000504,0.000250,0.000041,0.007239,0.000717,0.007934,0.005173,0.006053,0.000378,0.979566,K6QO
0_2,1.050081e-06,0.000030,1.271614e-06,0.000010,0.000009,3.174679e-07,0.000013,0.998910,0.000145,0.000022,0.000075,0.000017,0.000007,8.675750e-07,0.000240,0.000002,0.000041,0.000019,0.000384,0.000067,0.000000,0.000010,0.000037,0.000005,0.000005,0.000005,0.000005,0.000010,0.999904,0.000080,0.000034,0.000043,0.000017,0.000013,0.000005,0.000253,0.000033,0.000024,0.000020,0.000129,0.000036,0.000000,0.000175,0.001725,0.000048,0.000077,0.000097,0.000236,0.000170,0.958341,0.004721,0.001308,0.002570,0.000256,0.000177,0.000106,0.026640,0.000491,0.028669,0.029000,0.006985,0.000785,0.000000,RVSZ
0_3,6.748771e-06,0.000189,5.630120e-06,0.000017,0.000029,8.136376e-07,0.000039,0.000000,0.001783,0.000072,0.000286,0.000080,0.000025,1.899558e-06,0.012337,0.000038,0.000114,0.000053,0.004835,0.001153,0.978929,0.000096,0.000418,0.000033,0.000029,0.000036,0.000035,0.000068,0.000000,0.001277,0.000499,0.000561,0.000122,0.000093,0.000052,0.009613,0.000218,0.000195,0.000148,0.001819,0.000572,0.989596,0.000175,0.001725,0.000048,0.000077,0.000097,0.000236,0.000170,0.000000,0.004721,0.001308,0.002570,0.000256,0.000177,0.000106,0.026640,0.000491,0.028669,0.029000,0.006985,0.000785,0.953628,K6QO
0_4,1.410049e-05,0.013857,1.315864e-03,0.000134,0.018212,1.173993e-05,0.936572,0.000000,0.000284,0.000513,0.000150,0.000127,0.000005,3.967257e-06,0.000377,0.000015,0.000025,0.000020,0.000270,0.000000,0.027834,0.000056,0.060541,0.001550,0.001356,0.009913,0.000035,0.954791,0.000000,0.001736,0.001229,0.000260,0.000725,0.000324,0.000174,0.004196,0.000139,0.000120,0.000106,0.000737,0.000000,0.125487,0.000583,0.138840,0.011209,0.005714,0.027177,0.000124,0.910289,0.000000,0.011399,0.016217,0.002240,0.005876,0.002898,0.001154,0.019321,0.001078,0.003338,0.002592,0.003997,0.000000,0.837865,SOP4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4_13358,2.424181e-04,0.004970,3.186360e-05,0.000444,0.000499,4.866027e-06,0.000176,0.000000,0.002899,0.000826,0.001463,0.001008,0.000369,1.491695e-04,0.012714,0.000624,0.000299,0.000368,0.001166,0.010411,0.961295,0.000195,0.000283,0.000084,0.000150,0.000115,0.000091,0.000157,0.000000,0.000837,0.000983,0.003300,0.000378,0.000653,0

In [18]:
new_df_test = pd.DataFrame(np.zeros((test['ID'].nunique(), test['PCODE'].nunique()*3)))
new_df_test.index = test['ID'].unique()
new_df_test.columns = list(test['PCODE'].unique() + '_0') + list(test['PCODE'].unique() + '_1') + list(test['PCODE'].unique() + '_2') 

In [19]:
test

,Label1,Label2,Label3,ID,PCODE
0,0.000002,0.000106,0.000351,F86J5PC,P5DA
1,0.000395,0.000344,0.002852,F86J5PC,RIBP
2,0.000003,0.000065,0.000044,F86J5PC,8NN1
3,0.000071,0.000061,0.000097,F86J5PC,7POT
4,0.000073,0.000050,0.000036,F86J5PC,66FJ
...,...,...,...,...,...
197142,0.000008,0.000032,0.000425,WHII93K,JWFN
197143,0.000022,0.000019,0.012963,WHII93K,JZ9D
197144,0.000025,0.000014,0.008980,WHII93K,J9JW
197145,0.000174,0.000123,0.004435,WHII93K,GHYX


In [20]:
for v in tqdm(test.values):
   new_df_test.loc[v[3], v[4]+'_0'] = v[0]
   new_df_test.loc[v[3], v[4]+'_1'] = v[1]
   new_df_test.loc[v[3], v[4]+'_2'] = v[2]

100%|██████████| 197147/197147 [01:45<00:00, 1872.44it/s]


In [21]:
new_df_test

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,K6QO_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,RVSZ_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,K6QO_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,RVSZ_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,K6QO_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,RVSZ_2
F86J5PC,2.038956e-06,0.000395,3.067319e-06,0.000071,0.000073,1.408622e-06,0.000324,0.001754,0.000043,0.000104,0.000025,0.000041,5.788468e-06,0.981331,0.010432,1.969000e-04,0.000121,0.000117,0.002254,0.002703,0.000000,0.000106,0.000344,6.485056e-05,0.000061,4.950373e-05,0.000102,0.000136,0.000946,0.000468,0.000398,0.000137,0.000309,0.000309,0.991898,0.006118,0.000337,0.000286,0.000213,0.004326,0.004779,0.000000,0.000351,0.002852,0.000044,0.000097,0.000036,0.000366,0.000559,0.006109,0.000822,0.001211,0.000074,0.000364,0.000463,0.951794,0.017377,0.000754,0.014546,0.014503,0.021160,0.003330,0.000000
H6141K3,4.533728e-05,0.001618,1.338280e-04,0.000423,0.000722,5.054484e-06,0.000852,0.010072,0.000747,0.003395,0.001391,0.000605,2.223099e-04,0.000000,0.490467,1.635593e-01,0.000821,0.000804,0.148844,0.172166,0.000000,0.000561,0.001567,1.538794e-04,0.000618,6.337112e-04,0.000652,0.000227,0.009206,0.002240,0.001738,0.000361,0.001024,0.005218,0.000000,0.342929,0.020726,0.002023,0.001780,0.267824,0.189088,0.000000,0.000260,0.004020,0.000323,0.000314,0.000200,0.000983,0.001600,0.010724,0.001987,0.001267,0.000640,0.003344,0.005487,0.000000,0.446011,0.019027,0.017275,0.017942,0.243352,0.112068,0.000000
RBAYUXZ,3.768160e-05,0.002367,2.048250e-04,0.000194,0.000683,2.046647e-06,0.000811,0.006395,0.001105,0.006647,0.004289,0.000109,6.722619e-06,0.000000,0.879228,1.735700e-02,0.000544,0.000238,0.008176,0.070898,0.000000,0.000685,0.006903,1.087682e-03,0.000818,3.137980e-03,0.000290,0.002362,0.015840,0.003634,0.019590,0.002467,0.001954,0.000867,0.000000,0.789709,0.015551,0.001891,0.000849,0.019131,0.121261,0.000000,0.002669,0.026861,0.001377,0.004061,0.002561,0.000963,0.009362,0.049599,0.006476,0.021498,0.006989,0.004071,0.001825,0.000000,0.857083,0.012130,0.058791,0.066989,0.037354,0.077270,0.000000
KCBILBQ,1.977373e-06,0.000176,1.649559e-06,0.000035,0.000036,3.905146e-07,0.000191,0.000977,0.000152,0.000435,0.000251,0.000040,2.931731e-06,0.985894,0.005515,1.189624e-04,0.000081,0.000117,0.003064,0.002905,0.000000,0.000079,0.000245,3.854714e-05,0.000041,3.876904e-05,0.000029,0.000205,0.000814,0.000402,0.000518,0.000151,0.000153,0.000081,0.991719,0.004782,0.000163,0.000119,0.000090,0.002589,0.001698,0.000000,0.000251,0.001065,0.000053,0.000069,0.000046,0.000240,0.000456,0.002322,0.000963,0.001613,0.000444,0.000315,0.000182,0.953597,0.009350,0.000527,0.022857,0.018897,0.006625,0.001918,0.000000
LSEC1ZJ,1.948958e-06,0.000021,2.426809e-06,0.000018,0.000008,1.299198e-07,0.000006,0.000826,0.000058,0.000079,0.000040,0.000134,1.199630e-06,0.000002,0.001978,1.154888e-06,0.000000,0.993502,0.000441,0.000058,0.002366,0.000067,0.000243,9.046265e-05,0.000034,2.304211e-05,0.000067,0.000055,0.003442,0.000370,0.000232,0.000100,0.000436,0.000098,0.000022,0.011235,0.000741,0.000000,0.994480,0.001424,0.000538,0.004083,0.000698,0.001864,0.000071,0.000090,0.000085,0.000721,0.000275,0.046129,0.002771,0.002899,0.000192,0.000260,0.000415,0.116061,0.185898,0.002451,0.000000,0.840203,0.045734,0.002539,0.263557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0GMU5UH,4.688310e-06,0.000284,3.063728e-06,0.000091,0.000110,6.767243e-07,0.000054,0.001178,0.000519,0.003152,0.000069,0.000846,2.518867e-05,0.000010,0.000000,2.654164e-06,0.000000,0.979099,0.008910,0.000047,0.005170,0.000250,0.002113,1.348303e-04,0.000139,2.945262e-04,0.000126,0.000207

In [22]:
new_df

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,RVSZ_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,K6QO_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,RVSZ_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,K6QO_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,RVSZ_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,K6QO_2,target
0_0,2.062916e-06,0.000040,1.329303e-06,0.000007,0.000007,1.562166e-07,0.000004,0.999314,0.000062,0.000012,0.000108,0.000050,0.000030,9.034984e-07,0.000090,0.000008,0.000020,0.000016,0.000182,0.000037,0.000000,0.000007,0.000067,0.000004,0.000003,0.000003,0.000005,0.000007,0.999873,0.000066,0.000022,0.000036,0.000011,0.000010,0.000005,0.000160,0.000036,0.000022,0.000016,0.000163,0.000031,0.000000,0.000104,0.001975,0.000066,0.000071,0.000082,0.000173,0.000069,0.975555,0.001077,0.000423,0.001417,0.000504,0.000250,0.000041,0.007239,0.000717,0.007934,0.005173,0.006053,0.000378,0.000000,RVSZ
0_1,1.194800e-05,0.000173,3.812930e-06,0.000010,0.000021,3.163981e-07,0.000012,0.000000,0.000614,0.000043,0.000445,0.000158,0.000069,2.675490e-06,0.003556,0.000186,0.000047,0.000042,0.002238,0.000797,0.991568,0.000084,0.000554,0.000027,0.000027,0.000023,0.000041,0.000052,0.000000,0.001033,0.000398,0.000613,0.000078,0.000076,0.000056,0.004113,0.000281,0.000222,0.000148,0.004045,0.000471,0.997128,0.000104,0.001975,0.000066,0.000071,0.000082,0.000173,0.000069,0.000000,0.001077,0.000423,0.001417,0.000504,0.000250,0.000041,0.007239,0.000717,0.007934,0.005173,0.006053,0.000378,0.979566,K6QO
0_2,1.050081e-06,0.000030,1.271614e-06,0.000010,0.000009,3.174679e-07,0.000013,0.998910,0.000145,0.000022,0.000075,0.000017,0.000007,8.675750e-07,0.000240,0.000002,0.000041,0.000019,0.000384,0.000067,0.000000,0.000010,0.000037,0.000005,0.000005,0.000005,0.000005,0.000010,0.999904,0.000080,0.000034,0.000043,0.000017,0.000013,0.000005,0.000253,0.000033,0.000024,0.000020,0.000129,0.000036,0.000000,0.000175,0.001725,0.000048,0.000077,0.000097,0.000236,0.000170,0.958341,0.004721,0.001308,0.002570,0.000256,0.000177,0.000106,0.026640,0.000491,0.028669,0.029000,0.006985,0.000785,0.000000,RVSZ
0_3,6.748771e-06,0.000189,5.630120e-06,0.000017,0.000029,8.136376e-07,0.000039,0.000000,0.001783,0.000072,0.000286,0.000080,0.000025,1.899558e-06,0.012337,0.000038,0.000114,0.000053,0.004835,0.001153,0.978929,0.000096,0.000418,0.000033,0.000029,0.000036,0.000035,0.000068,0.000000,0.001277,0.000499,0.000561,0.000122,0.000093,0.000052,0.009613,0.000218,0.000195,0.000148,0.001819,0.000572,0.989596,0.000175,0.001725,0.000048,0.000077,0.000097,0.000236,0.000170,0.000000,0.004721,0.001308,0.002570,0.000256,0.000177,0.000106,0.026640,0.000491,0.028669,0.029000,0.006985,0.000785,0.953628,K6QO
0_4,1.410049e-05,0.013857,1.315864e-03,0.000134,0.018212,1.173993e-05,0.936572,0.000000,0.000284,0.000513,0.000150,0.000127,0.000005,3.967257e-06,0.000377,0.000015,0.000025,0.000020,0.000270,0.000000,0.027834,0.000056,0.060541,0.001550,0.001356,0.009913,0.000035,0.954791,0.000000,0.001736,0.001229,0.000260,0.000725,0.000324,0.000174,0.004196,0.000139,0.000120,0.000106,0.000737,0.000000,0.125487,0.000583,0.138840,0.011209,0.005714,0.027177,0.000124,0.910289,0.000000,0.011399,0.016217,0.002240,0.005876,0.002898,0.001154,0.019321,0.001078,0.003338,0.002592,0.003997,0.000000,0.837865,SOP4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4_13358,2.424181e-04,0.004970,3.186360e-05,0.000444,0.000499,4.866027e-06,0.000176,0.000000,0.002899,0.000826,0.001463,0.001008,0.000369,1.491695e-04,0.012714,0.000624,0.000299,0.000368,0.001166,0.010411,0.961295,0.000195,0.000283,0.000084,0.000150,0.000115,0.000091,0.000157,0.000000,0.000837,0.000983,0.003300,0.000378,0.000653,0

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(new_df['target'])
le.transform(new_df['target'])

array([19, 12, 19, ..., 12, 19, 12])

In [24]:
new_df['target'] = le.transform(new_df['target'])

In [25]:
from catboost import CatBoostClassifier

models = []
models.append(CatBoostClassifier(random_state=17, max_depth = 3))
models.append(CatBoostClassifier(random_state=18, num_trees=700, max_depth=5))
models.append(CatBoostClassifier(random_state=19, num_trees=500, max_depth = 3))
models.append(CatBoostClassifier(random_state=20, num_trees=1200, max_depth=3))
#models.append(CatBoostClassifier(random_state=21))

In [26]:
for i in range(len(models)):
  models[i].fit(new_df.drop(columns=['target',]), new_df['target'])

Learning rate set to 0.098003
0:	learn: 1.7907489	total: 461ms	remaining: 7m 41s
1:	learn: 1.5778374	total: 844ms	remaining: 7m 1s
2:	learn: 1.4281310	total: 1.23s	remaining: 6m 48s
3:	learn: 1.3142063	total: 1.61s	remaining: 6m 40s
4:	learn: 1.2118985	total: 2s	remaining: 6m 37s
5:	learn: 1.1332064	total: 2.38s	remaining: 6m 34s
6:	learn: 1.0724756	total: 2.77s	remaining: 6m 32s
7:	learn: 1.0170562	total: 3.17s	remaining: 6m 32s
8:	learn: 0.9629786	total: 3.58s	remaining: 6m 33s
9:	learn: 0.9238660	total: 3.96s	remaining: 6m 32s
10:	learn: 0.8884107	total: 4.37s	remaining: 6m 33s
11:	learn: 0.8577515	total: 4.73s	remaining: 6m 29s
12:	learn: 0.8280643	total: 5.06s	remaining: 6m 23s
13:	learn: 0.7944170	total: 5.4s	remaining: 6m 20s
14:	learn: 0.7673458	total: 5.75s	remaining: 6m 17s
15:	learn: 0.7464041	total: 6.09s	remaining: 6m 14s
16:	learn: 0.7279446	total: 6.43s	remaining: 6m 11s
17:	learn: 0.7097797	total: 6.75s	remaining: 6m 8s
18:	learn: 0.6886221	total: 7.08s	remaining: 6m 5s

In [27]:
df_answ = []
for num_pred in range(len(models)):
  proba = models[num_pred].predict_proba(new_df_test)
  y_test = pd.DataFrame(proba)
  y_test.columns = le.inverse_transform(y_test.columns)

  true_values = []
  for v in test_.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test_.columns[8:][index]:
      true_values.append(v[0] + ' X ' + k)

  answer_mass = []
  for i in range(new_df_test.index.shape[0]):
    id = new_df_test.index[i]
    for c in y_test.columns:
      answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

  df_answer = pd.DataFrame(answer_mass)
  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answ.append(df_answer)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
df_answ

[            ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000061
 1       F86J5PC X 7POT  0.000023
 2       F86J5PC X 8NN1  0.000008
 3       F86J5PC X AHXO  0.000079
 4       F86J5PC X BSTQ  0.000007
 ...                ...       ...
 209995  WHII93K X PYUQ  0.000087
 209996  WHII93K X QBOL  0.000190
 209997  WHII93K X RIBP  0.000019
 209998  WHII93K X RVSZ  0.999532
 209999  WHII93K X SOP4  0.000005
 
 [210000 rows x 2 columns],             ID X PCODE         Label
 0       F86J5PC X 66FJ  2.355799e-05
 1       F86J5PC X 7POT  2.755510e-06
 2       F86J5PC X 8NN1  5.820439e-07
 3       F86J5PC X AHXO  3.050135e-05
 4       F86J5PC X BSTQ  1.198837e-06
 ...                ...           ...
 209995  WHII93K X PYUQ  1.736253e-05
 209996  WHII93K X QBOL  9.503058e-05
 209997  WHII93K X RIBP  3.839140e-06
 209998  WHII93K X RVSZ  9.998276e-01
 209999  WHII93K X SOP4  1.618947e-06
 
 [210000 rows x 2 columns],             ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000043
 1       F

In [29]:
df_answ_labels = []
for v in df_answ:
  df_answ_labels.append(v['Label'])

In [30]:
df_answ_labels

[0         0.000061
 1         0.000023
 2         0.000008
 3         0.000079
 4         0.000007
             ...   
 209995    0.000087
 209996    0.000190
 209997    0.000019
 209998    0.999532
 209999    0.000005
 Name: Label, Length: 210000, dtype: float64, 0         2.355799e-05
 1         2.755510e-06
 2         5.820439e-07
 3         3.050135e-05
 4         1.198837e-06
               ...     
 209995    1.736253e-05
 209996    9.503058e-05
 209997    3.839140e-06
 209998    9.998276e-01
 209999    1.618947e-06
 Name: Label, Length: 210000, dtype: float64, 0         0.000043
 1         0.000019
 2         0.000009
 3         0.000063
 4         0.000007
             ...   
 209995    0.000095
 209996    0.000280
 209997    0.000018
 209998    0.999390
 209999    0.000015
 Name: Label, Length: 210000, dtype: float64, 0         0.000055
 1         0.000020
 2         0.000006
 3         0.000054
 4         0.000008
             ...   
 209995    0.000068
 209996    0.000183
 

In [31]:
np.mean(df_answ_labels, axis=0)

array([4.54629078e-05, 1.61103442e-05, 5.68418957e-06, ...,
       1.48626630e-05, 9.99571474e-01, 7.26858890e-06])

In [32]:
df_answer = df_answ[0]
df_answer['Label'] = np.mean(df_answ_labels, axis=0)

In [33]:
df_answer

,ID X PCODE,Label
0,F86J5PC X 66FJ,0.000045
1,F86J5PC X 7POT,0.000016
2,F86J5PC X 8NN1,0.000006
3,F86J5PC X AHXO,0.000057
4,F86J5PC X BSTQ,0.000006
...,...,...
209995,WHII93K X PYUQ,0.000067
209996,WHII93K X QBOL,0.000187
209997,WHII93K X RIBP,0.000015
209998,WHII93K X RVSZ,0.999571


In [34]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('finale_version_multiClass_stack.csv', index=False)

In [35]:
from google.colab import files
files.download('finale_version_multiClass_stack.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OK